In [ ]:
# Query, open
# Filter for clouds
#    if cloudy, query n+ith day
#    else continue
# Ensure query time is good
# If so, continue, else, wait.

In [1]:
def next_day(date):
    day = '0' + str(int(date[8:]) + 1)
    if len(day) > 2:
        day = day[1:]
    return date[:8] + day

def is_cloudy(img):
    #Takes Image object, returns bool if it meets 'cloudy' criteria
    k = 75 #Threshold coeff.
    
    hist = img.histogram()
    Rm = np.mean(hist[:256])
    Gm = np.mean(hist[256:512])
    Bm = np.mean(hist[512:])
    W = [hist[255], hist[511], hist[-1]]
    
    if W[0] > k*Rm or W[1] > k*Gm or W[2] > k*Bm:
        return True
    else:
        return False

In [2]:
import os, cv2, requests, base64, imageio
from PIL import Image
from io import BytesIO
import seaborn as sn
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#Load lat and long csv
us_cities = pd.read_csv("uscities.csv")
ut_cities = us_cities.loc[us_cities['state_name'] == "Utah"]
ut_cities = ut_cities.reset_index(drop=True)

#n_mo = 5*12 #Total months to query
n_mo = 1
start = pd.Timestamp('2020-04-06T12')
dates = [str(start - pd.Timedelta(days=30*_))[:10] for _ in range(n_mo)]
print(dates)

site = "https://api.nasa.gov/planetary/earth/imagery?"
key1 = "II3XUxmGYfFKmyapNeXjrH3Os3ILNhQScjW6bbMT"
key2 = "07b9v4Pg29fysvhDf1q6x0UyOaGZxXIfryIv9PKO"

queried = []
counter = 0
for date in dates:
    for i, city in ut_cities.iterrows():
        #Exclude cities with an even index
        #if i % 2 != 0: 
        if i % 70 == 0:
            date_ = date
            
            #Query API for city image
            url = site + "lon=" + str(city['lng']) + "&lat=" + str(city['lat']) + \
                "&dim=.2&date=" + date + "&api_key=" + key1
            response = requests.get(url)
            
            print("Ok?",response.ok,'i =',i)
            if response.ok:
                img = Image.open(BytesIO(response.content))
                img = img.resize((720,720))
                cloudy = is_cloudy(img)
                
                attempts = 0
                while cloudy and attempts < 5:
                    #NOTE - Selects nearest date, which remains constant for some images
                    #May need better target locations and larger dt for attempts.
                    date = next_day(date)
                    url = site + "lon=" + str(city['lng']) + "&lat=" + str(city['lat']) + \
                        "&dim=.2&date=" + date + "&api_key=" + key1
                    response = requests.get(url)
                    
                    img = Image.open(BytesIO(response.content))
                    img = img.resize((720,720))
                    cloudy = is_cloudy(img)
                    attempts += 1
                    
            if response.ok:
                name = city['city'].replace(' ','_') + '_' + date.replace('-','_') + '.png'
                img.save(name)
            date = date_               
            counter += 1
print(counter)

['2020-04-06']
Ok? True i = 0
Ok? True i = 70
Ok? True i = 140
Ok? True i = 210
Ok? True i = 280
5


In [ ]:
image_test = Image.open('slc_2020_04_26.png')

In [4]:
# 29 counties in Utah - selectiong first 20 cities
ut_cities

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,Salt Lake City,Salt Lake City,UT,Utah,49035,Salt Lake,40.7777,-111.9306,1098526,699,polygon,False,True,America/Denver,1,84116 84115 84113 84112 84111 84144 84128 8418...,1840021383
1,Ogden,Ogden,UT,Utah,49057,Weber,41.2278,-111.9682,578523,1230,polygon,False,True,America/Denver,2,84404 84401 84403 84244 84407 84408 84409 84415,1840020135
2,Provo,Provo,UT,Utah,49049,Utah,40.2457,-111.6457,500617,1080,polygon,False,True,America/Denver,2,84604 84606 84601 84602 84603 84605,1840020174
3,West Valley City,West Valley City,UT,Utah,49035,Salt Lake,40.6889,-112.0115,135248,1472,polygon,False,True,America/Denver,2,84119 84120 84128 84118 84125 84126 84127 8412...,1840021389
4,St. George,St. George,UT,Utah,49053,Washington,37.0770,-113.5770,120899,449,polygon,False,True,America/Denver,2,84770 84790 84791,1840021559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,Echo,Echo,UT,Utah,49043,Summit,40.9803,-111.4393,49,23,polygon,False,False,America/Denver,3,84024,1840024183
318,Clear Creek,Clear Creek,UT,Utah,49007,Carbon,39.6429,-111.1547,37,69,polygon,False,False,America/Denver,3,84526,1840024370
319,Sutherland,Sutherland,UT,Utah,49027,Millard,39.3896,-112.6400,34,6,polygon,False,False,America/Denver,3,84624,1840024465
320,Oasis,Oasis,UT,Utah,49027,Millard,39.2923,-112.6290,28,28,polygon,False,False,America/Denver,3,84624,1840024464
